In [1]:
# %reset -f

import sys
import os
import joblib
import matplotlib
import numpy as np
import pandas as pd 
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score, median_absolute_error

# install custom package with: pip install git+https://github.com/owenodriscoll/AutoML.git
from AutoML import AutomatedRegression

/export/home/owen/anaconda3/envs/env_paper/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
sys.path.insert(0, os.path.abspath('..'))  # hacky way to import within package
from src import equations_misc as eqm

In [3]:
#%% load data

data_dir = '/export/home/owen/Documents/scripts/SAR_paper/data/' # < -- change data_dir to custom location
df = pd.read_csv(f'{data_dir}df_rolls_buoy.csv')

In [4]:
#%% filtering of estimates 

# -- conditions for filtering
window_effect_threshhold_low = 0.5
window_effect_threshhold_high = 1.5
S_windfield_xi_norm_std_threshhold = 0.25
inertial_subrange_length_fraction_threshold = 0.2 
inertial_subrange_length_threshold = 200 # m
peak_max_threshold = 2950
peak_min_threshold = 610
min_heatflux = 0 # (into atmos)
max_obukhov = 0

# -- filtering
df_val = df.loc[(df['L_era5'] < max_obukhov) & (df['hsb_era5'] > min_heatflux) & (df['L_buoy'] < max_obukhov) & (df['hsb_buoy'] > min_heatflux)]
df_val = df_val[(df_val.window_effect >= window_effect_threshhold_low) & (df_val.window_effect <= window_effect_threshhold_high)]
df_val = df_val[df_val.S_windfield_xi_norm_std <= S_windfield_xi_norm_std_threshhold]
df_val = df_val[(df_val.spectral_peak - df_val.spectral_valley)/df_val.spectral_peak >= inertial_subrange_length_fraction_threshold]
df_val = df_val[((df_val.spectral_peak - df_val.spectral_valley) >= inertial_subrange_length_threshold)]
df_val = df_val[(df_val.spectral_peak< peak_max_threshold) & (df_val.spectral_peak > peak_min_threshold)]

# -- outlier removal between parameters 'window_effect' and 'S_windfield_xi_norm_std'
df_val = eqm.outlier_detector(df_val, 'window_effect', 'S_sigma0_xi_norm_std', pca_comp=0.80, neighbours = 100, plot_PCA=False)    # S_normalised_deviation, sign_wind2
df_val = df_val.reset_index(drop = True)  

# -- select observation data only
keep_after_index = list(df_val.keys()).index('U_n') # U_n is first observation column
keep_before_index = list(df_val.keys()).index('energy_dir_range') +1 # S_windfield_xi_norm_std is the last measured param 
df_obs = df_val.iloc[:,keep_after_index:keep_before_index] 

# -- replace original estimate of Obukhov length by the a logarithmic version 
df_obs['L'] = np.log10(abs(df_obs['L']))

In [20]:
#%% Select data, split into X and y- matrix and exclude scenes for analysis later on

# -- observation
X_sar = df_obs.drop(columns = ['S_sigma0_xi_norm_std', 'S_sigma0_xi_mean'])

# -- select all the buoy observations (including )
X_buoy = df_val[[param for param in list(df_val.keys()) if 'buoy' in param or 'depth' in param]]
X_buoy.drop(columns = ['buoy', 'depth_pres', 'depth_curr', 'dtime_train_buoy', 'lat_buoy', 
                       'pres_buoy', 'lon_buoy', 'time_buoy', 'cspd_buoy', 'cdir_buoy', 
                       'lwav_buoy', 'swav_buoy', 'pblh_buoy'], inplace = True)

# -- validation
y_era5 = np.log10(abs(df_val['L_era5'])) 

/tmp/ipykernel_158379/540422231.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_buoy.drop(columns = ['buoy', 'depth_pres', 'depth_curr', 'dtime_train_buoy', 'lat_buoy',


In [27]:
#%% Prepare Machine Learning 

sar_to_era5 = AutomatedRegression(
    y = y_era5, 
    X = X_sar, 
    test_frac = 0.2, 
    timeout = 300,  # currently times out after 300 seconds, for study set to 2400
    n_trial = 20,   # currently allowed a max of 20 trials, for study set to 200
    metric_optimise = median_absolute_error,
    metric_assess = [mean_squared_error, r2_score, explained_variance_score, median_absolute_error],
    reload_study = False,
    optimisation_direction = 'minimize', 
    write_folder = f'{data_dir}/examp_rolls_buoy_regression_sar_to_era5',
    models_to_optimize = ['lightgbm', 'catboost', 'xgboost','bayesianridge', 'lassolars'],
    random_state = 42)


era5_to_buoy = AutomatedRegression(
    y = y_era5, 
    X = X_buoy, 
    test_frac = 0.2, 
    timeout = 300,  # currently times out after 300 seconds, for study set to 2400
    n_trial = 20,   # currently allowed a max of 20 trials, for study set to 200
    metric_optimise = median_absolute_error,
    metric_assess = [mean_squared_error, r2_score, explained_variance_score, median_absolute_error],
    reload_study = False,
    optimisation_direction = 'minimize', 
    write_folder = f'{data_dir}/examp_rolls_buoy_regression_era5_to_buoy',
    models_to_optimize = ['lightgbm', 'catboost', 'xgboost','bayesianridge', 'lassolars'],
    random_state = 42)

In [28]:
#%% Apply the regression

sar_to_era5.apply()
era5_to_buoy.apply()

[I 2023-03-29 15:47:04,838] A new study created in RDB with name: regression_lightgbm
[I 2023-03-29 15:47:05,038] Trial 0 finished with value: 0.19134453675771856 and parameters: {'scaler': 'minmax', 'objective': 'regression', 'max_depth': 5, 'n_estimators': 88, 'max_bin': 127, 'min_gain_to_split': 14.548647782429915, 'lambda_l1': 0.31044435499483225, 'lambda_l2': 8.148018307012941e-07, 'num_leaves': 48, 'feature_fraction': 0.2650640588680905, 'bagging_fraction': 0.373818018663584, 'bagging_freq': 4, 'min_child_samples': 44, 'random_state': 42, 'verbosity': -1, 'transformers': 'quantile_trans', 'n_quantiles': 600}. Best is trial 0 with value: 0.19134453675771856.
[I 2023-03-29 15:47:06,477] Trial 1 finished with value: 0.19057352012364376 and parameters: {'scaler': 'robust', 'objective': 'regression', 'max_depth': 6, 'n_estimators': 332, 'max_bin': 255, 'min_gain_to_split': 14.233283058799998, 'lambda_l1': 4.905556676028774, 'lambda_l2': 0.18861495878553936, 'num_leaves': 79, 'feature_

[I 2023-03-29 15:47:32,059] Trial 16 finished with value: 0.1585107861342948 and parameters: {'scaler': 'robust', 'objective': 'regression', 'max_depth': 20, 'n_estimators': 334, 'max_bin': 127, 'min_gain_to_split': 0.38026115123186266, 'lambda_l1': 4.611431184204183, 'lambda_l2': 0.33405739074934476, 'num_leaves': 179, 'feature_fraction': 0.46805764997284294, 'bagging_fraction': 0.2559648880637612, 'bagging_freq': 2, 'min_child_samples': 26, 'random_state': 42, 'verbosity': -1, 'transformers': 'quantile_trans', 'n_quantiles': 2700}. Best is trial 15 with value: 0.1338248038662146.
[I 2023-03-29 15:47:33,019] Trial 17 finished with value: 0.15027134604476328 and parameters: {'scaler': 'minmax', 'objective': 'regression', 'max_depth': 14, 'n_estimators': 234, 'max_bin': 255, 'min_gain_to_split': 0.6900396303262912, 'lambda_l1': 2.3257296032850187e-08, 'lambda_l2': 0.5002018067635705, 'num_leaves': 181, 'feature_fraction': 0.5267564461785927, 'bagging_fraction': 0.18805074458590132, 'bag

[I 2023-03-29 15:51:25,272] Trial 13 finished with value: 0.14040596332102656 and parameters: {'scaler': 'minmax', 'booster': 'gbtree', 'lambda': 0.00030571021793716305, 'alpha': 5.892899649588588e-05, 'random_state': 42, 'verbosity': 0, 'max_depth': 13, 'n_estimators': 85, 'eta': 0.14493614324797113, 'min_child_weight': 6.322642879195303, 'gamma': 0.13094456588333636, 'subsample': 0.6971836348150395, 'colsample_bytree': 0.2602323701827763, 'max_bin': 1024, 'transformers': 'quantile_trans', 'n_quantiles': 300}. Best is trial 13 with value: 0.14040596332102656.
[I 2023-03-29 15:51:25,518] Trial 14 pruned. 
[I 2023-03-29 15:51:25,746] Trial 15 pruned. 
[I 2023-03-29 15:51:26,007] Trial 16 pruned. 
[I 2023-03-29 15:51:26,871] Trial 17 pruned. 
[I 2023-03-29 15:51:27,950] Trial 18 finished with value: 0.15800192656652307 and parameters: {'scaler': 'robust', 'booster': 'gblinear', 'lambda': 1.7482833190810473e-07, 'alpha': 6.796810323010441e-07, 'random_state': 42, 'verbosity': 0, 'transfor

[I 2023-03-29 15:52:00,422] Trial 2 finished with value: 0.14811541363146968 and parameters: {'scaler': None, 'objective': 'regression', 'max_depth': 12, 'n_estimators': 374, 'max_bin': 127, 'min_gain_to_split': 8.968499682166277, 'lambda_l1': 1.9809253750493907, 'lambda_l2': 6.257956190096665e-08, 'num_leaves': 51, 'feature_fraction': 0.14070456001948428, 'bagging_fraction': 0.39279729768693794, 'bagging_freq': 3, 'min_child_samples': 28, 'random_state': 42, 'verbosity': -1, 'transformers': None}. Best is trial 1 with value: 0.14217189748927378.
[I 2023-03-29 15:52:04,134] Trial 3 finished with value: 0.14520651574052337 and parameters: {'scaler': 'robust', 'objective': 'regression', 'max_depth': 4, 'n_estimators': 1906, 'max_bin': 511, 'min_gain_to_split': 10.93510752061481, 'lambda_l1': 0.08738424135626986, 'lambda_l2': 4.638759594322625e-08, 'num_leaves': 93, 'feature_fraction': 0.20428215357261675, 'bagging_fraction': 0.8767930832880342, 'bagging_freq': 5, 'min_child_samples': 34,

[I 2023-03-29 15:52:28,735] Trial 1 finished with value: 0.11734856184636173 and parameters: {'scaler': 'robust', 'depth': 6, 'iterations': 237, 'learning_rate': 0.02725296361559705, 'l2_leaf_reg': 4.859254947164246, 'rsm': 0.3953096619468215, 'logging_level': 'Silent', 'random_seed': 42, 'transformers': 'quantile_trans', 'n_quantiles': 400}. Best is trial 0 with value: 0.09912881215222276.
[I 2023-03-29 15:52:30,213] Trial 2 pruned. 
[I 2023-03-29 15:52:34,041] Trial 3 finished with value: 0.10327406055971806 and parameters: {'scaler': 'robust', 'depth': 5, 'iterations': 348, 'learning_rate': 0.011291328372260002, 'l2_leaf_reg': 0.022449209314333803, 'rsm': 0.1515617652323075, 'logging_level': 'Silent', 'random_seed': 42, 'transformers': None}. Best is trial 0 with value: 0.09912881215222276.
[I 2023-03-29 15:53:09,610] Trial 4 finished with value: 0.10589665268156725 and parameters: {'scaler': 'standard', 'depth': 9, 'iterations': 135, 'learning_rate': 0.014544458649908071, 'l2_leaf_

[I 2023-03-29 15:55:33,191] Trial 5 pruned. 
[I 2023-03-29 15:55:33,750] Trial 6 finished with value: 0.10171716820157159 and parameters: {'scaler': 'robust', 'n_iter': 42, 'tol': 60.3152109506073, 'alpha_1': 2.8411299402417385e-06, 'alpha_2': 7.815488716572756e-05, 'lambda_1': 7.707923003240884e-06, 'lambda_2': 3.603521890624151e-05, 'transformers': None}. Best is trial 0 with value: 0.10065841501844816.
[I 2023-03-29 15:55:34,358] Trial 7 finished with value: 0.11322738774084229 and parameters: {'scaler': 'robust', 'n_iter': 296, 'tol': 21.494037866926273, 'alpha_1': 2.05036580654678e-08, 'alpha_2': 4.194116049012003e-06, 'lambda_1': 0.00892851653002802, 'lambda_2': 3.267820063127127e-08, 'transformers': 'quantile_trans', 'n_quantiles': 1400}. Best is trial 0 with value: 0.10065841501844816.
[I 2023-03-29 15:55:34,690] Trial 8 finished with value: 0.1017359914823009 and parameters: {'scaler': None, 'n_iter': 203, 'tol': 68.84023964588958, 'alpha_1': 0.00022298383788118986, 'alpha_2':

In [32]:
#%% load previous study
stacking_regressor = joblib.load(f"{data_dir}/examp_rolls_buoy_regression_sar_to_era5/" + "stacked_model.joblib")
pred = stacking_regressor.predict(sar_to_era5.X_test)

r2 = r2_score(sar_to_era5.y_test, pred)
mae = median_absolute_error(sar_to_era5.y_test, pred)
print(f"SAR to ERA5 over buoys: Model explains {np.round(r2,3)*100}% of variance with a Median Absolute Error of {np.round(mae,3)} (on a log scale)")

#%% load previous study
stacking_regressor = joblib.load(f"{data_dir}/examp_rolls_buoy_regression_era5_to_buoy/" + "stacked_model.joblib")
pred = stacking_regressor.predict(era5_to_buoy.X_test)

r2 = r2_score(era5_to_buoy.y_test, pred)
mae = median_absolute_error(era5_to_buoy.y_test, pred)
print(f"Buoys to ERA5 over buoys: Model explains {np.round(r2,3)*100}% of variance with a Median Absolute Error of {np.round(mae,3)} (on a log scale)")

SAR to ERA5 over buoys: Model explains 43.0% of variance with a Median Absolute Error of 0.126 (on a log scale)
Buoys to ERA5 over buoys: Model explains 72.5% of variance with a Median Absolute Error of 0.103 (on a log scale)
